# JS - Partial Least Squares + Gradient Boosted Trees

In [ ]:
import joblib
import json
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_decomposition import PLSRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

In [ ]:
# number of PLS components to keep
COMPONENTS = 40

# parameters to try for models
# n_estimators = number of consecutive boosters
# max_depth = maximum depth of each tree
# learning_rate = shrinkage
# subsample = fraction of samples per tree
# colsample_bytree = fraction of features per tree
# reg_alpha = regularization on weights
# reg_lambda = l2 regularization on weights
PARAMS = {"n_estimators": [200],
          "max_depth": [6, 7, 8],
          "learning_rate": [0.05, 0.1, 0.15],
          "subsample": [0.7, 0.8, 0.9],
          "colsample_bytree": [0.7, 0.8, 0.9],
          "reg_alpha": [80.0, 100.0, 120.0],
          #"reg_lambda": [1.0, 10.0, 50.0]
         }

In [ ]:
# read data as 32 bit floats
file = os.path.join(os.pardir, "input", "jane-street-market-prediction", "train.csv")
dtype = {c: np.float32 for c in pd.read_csv(file, nrows=1).columns}
full_df = pd.read_csv(file, engine="c", dtype=dtype)

# split into training and validation
train_df = full_df[full_df["date"].between(86, 375)]
valid_df = full_df[full_df["date"].between(425, 500)]

# build features and labels
features = [f"feature_{x}" for x in range(130)]
train_X = train_df[features].to_numpy()
valid_X = valid_df[features].to_numpy()
train_y = train_df["resp"].gt(0.0).astype(np.float32).to_numpy()
valid_y = valid_df["resp"].gt(0.0).astype(np.float32).to_numpy()

In [ ]:
# preprocessing pipeline: impute with mean,
# z-score, project onto PLS features
si = SimpleImputer(strategy="mean")
train_X = si.fit_transform(train_X)

ss = StandardScaler()
train_X = ss.fit_transform(train_X)

pls = PLSRegression(n_components=COMPONENTS)
pls.fit(train_X, train_y) #(train_df["resp"] - train_df["resp"].mean()) / train_df["resp"].std())
train_X = pls.transform(train_X)

pp = make_pipeline(si, ss, pls)
valid_X = pp.transform(valid_X)

In [ ]:
# base model
clf = XGBClassifier(tree_method="gpu_hist", random_state=13)

# setup the data for parameter search
full_X = np.vstack([train_X, valid_X])
full_y = np.append(train_y, valid_y)
test_fold = np.append(np.full(train_X.shape[0], -1, dtype=np.int), np.zeros(valid_X.shape[0], dtype=np.int))
ps = PredefinedSplit(test_fold)

# find the parameters with best AUC score
cv = GridSearchCV(clf, PARAMS, cv=ps, scoring="roc_auc", refit=False)
cv.fit(full_X, full_y)

# show the best parameters
print(f"Highest AUC = {round(cv.best_score_, 4)} with parameters:")
for k, v in cv.best_params_.items():
    print(f"  {k} = {v}")

# need to refit the best model manually params, since the
# built-in refitting will refit on full_X, full_y
model = XGBClassifier(tree_method="gpu_hist", random_state=13, **cv.best_params_)
model.fit(train_X, train_y, verbose=False)

In [ ]:
probs = model.predict_proba(valid_X)
probs = probs[:, 1]

# precision vs recall
precisions, recalls, thresholds = precision_recall_curve(valid_y, probs)

plt.figure(figsize=(8, 5))
plt.plot(thresholds, precisions[:-1], "tab:blue", label="Precision")
plt.plot(thresholds, recalls[:-1], "tab:orange", label="Recall")
plt.legend()
plt.xlabel("Threshold")
plt.title("Validation Precision/recall at threshold")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(recalls, precisions, "tab:blue")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Validation  Precision at recall")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

# ROC curve
false_positives, true_positives, thresholds = roc_curve(valid_y, probs)
plt.figure(figsize=(8, 5))
plt.plot(false_positives, true_positives, "tab:blue")
plt.plot([0, 1], [0, 1], "tab:gray")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("Validation ROC curve")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

# AUC
print(f"Validation AUC: {roc_auc_score(valid_y, probs)}")

In [ ]:
probs = model.predict_proba(train_X)
probs = probs[:, 1]

# precision vs recall
precisions, recalls, thresholds = precision_recall_curve(train_y, probs)

plt.figure(figsize=(8, 5))
plt.plot(thresholds, precisions[:-1], "tab:blue", label="Precision")
plt.plot(thresholds, recalls[:-1], "tab:orange", label="Recall")
plt.legend()
plt.xlabel("Threshold")
plt.title("Training Precision/recall at threshold")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(recalls, precisions, "tab:blue")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Training  Precision at recall")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

# ROC curve
false_positives, true_positives, thresholds = roc_curve(train_y, probs)
plt.figure(figsize=(8, 5))
plt.plot(false_positives, true_positives, "tab:blue")
plt.plot([0, 1], [0, 1], "tab:gray")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("Training ROC curve")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

# AUC
print(f"Training AUC: {roc_auc_score(train_y, probs)}")

In [ ]:
# save model
joblib.dump(pp, "preprocessor.pkl")
model.save_model("model.xgb")